In [2]:
import re,requests, random
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import time
from selenium import webdriver
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD, PCA
import eli5
from IPython.display import display
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


### Question 1

In [177]:
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    lr = LinearRegression()
    lr.fit(X_train_dtm, y_train)
    y_pred_class = lr.predict(X_test_dtm)
    print('Accuracy: ', lr.score(X_test_dtm,y_test))
    display(eli5.show_weights(lr,feature_names=vect.get_feature_names()))
    print(vect.get_feature_names())

In [4]:
def tchange(text):
    text = "'"+text.replace('=',"':")
    return print(text)

In [90]:
def salaryConvOld(text):
    if text is None:
        return None
    elif any(x in text.lower() for x in ['day','hour','month','p/d','ph','p/h''p.d','p.h','bonus','daily']):
        return None
    else:
        t = text.split('$')[1]
        t = t.split('-')[0].strip()
        t = re.sub(r'([$,])','',t)
        t = t.lower().replace('k','000')
        try:
            t = re.search(r'\d+',t).group()
        except:
            return None
        if len(t)<4:
            return None
        else:
            return t

In [271]:
re.search(r'\d+','3242sfsafsa5').group()

'3242'

In [233]:
def sConv(text):
    if text is not None:
        
        m = re.search(r'$(\d+)',text.replace(',',''))
        return m
    

In [216]:
salaryConv('$130k')

'130000'

In [195]:
def extractor(keyword):
    random.seed()
    keyword = keyword.strip().replace(' ','-')
    dic = defaultdict(list)
    
    while True:
        if len(dic) == 0:
            html = requests.get(f'https://www.seek.com.au/{keyword}-jobs').text
        else:
            html = requests.get(f'https://www.seek.com.au/{address}').text

        soup = BeautifulSoup(html,'lxml')

        a = soup.find_all('a',{'data-automation':"jobTitle"})

        for title in a:
            dic['Titles'].append(title.text)
            
            link = title.get('href')

            html = requests.get(f'https://www.seek.com.au/{link}').text
            
            sp = BeautifulSoup(html,'lxml')
            
            b = sp.find('div',{'class':'_2e4Pi2B'})
            
            dic['Description'].append(b.text)
            
            loca = sp.find_all('dd')
            
            ll=[]
            
            for ele in loca:
                if ele.text not in ll:
                    ll.append(ele.text)

           # ll = list({ele.text for ele in loca})
            s = False
            for l in ll:
                if '$' in l:
                    dic['Salary'].append(l)
                    s = True
            if not s:
                dic['Salary'].append(None)
            print(ll)

            dic['Info'].append(ll)
            
            time.sleep(random.random())
        try:
            address = soup.find('a',{'data-automation':'page-next'}).get('href')
        except:
            break
        
    return pd.DataFrame(dic)

In [223]:
ms = extractor('data start')

['3 Feb 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Full Time', 'Information & Communication Technology, Database Development & Administration']
['6 Feb 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', '$55,000 - $59,999', 'Full Time', 'Information & Communication Technology, Consultants']
['7 Feb 2020', 'Sydney, Parramatta & Western Suburbs', '$26 per hour', 'Contract/Temp', 'Administration & Office Support, Data Entry & Word Processing']
['6 Feb 2020', 'Melbourne, Eastern Suburbs', 'Competitive rate', 'Contract/Temp', 'Information & Communication Technology, Database Development & Administration']
['10 Feb 2020', 'Perth, CBD, Inner & Western Suburbs', 'Casual/Vacation', 'Administration & Office Support, Data Entry & Word Processing']
['10 Feb 2020', 'Sydney', 'Contract/Temp', 'Information & Communication Technology, Business/Systems Analysts']
['9 Feb 2020', 'ACT', 'Contract/Temp', 'Information & Communication Technology, Business/Systems Analysts']
['5 Feb 2020', 'Perth,

['30 Jan 2020', 'ACT', 'Contract/Temp', 'Information & Communication Technology, Business/Systems Analysts']
['24 Jan 2020', 'Sydney, Parramatta & Western Suburbs', 'Salary of $27.50 / hour ', 'Full Time', 'Trades & Services, Automotive Trades']
['30 Jan 2020', 'Perth, Fremantle & Southern Suburbs', 'Contract/Temp', 'Information & Communication Technology, Other']
['1 Feb 2020', 'Melbourne, Western Suburbs', 'Permanent Position. Best Conditions', 'Full Time', 'Manufacturing, Transport & Logistics, Warehousing, Storage & Distribution']
['6 Feb 2020', 'ACT', 'Open to Quote', 'Contract/Temp', 'Information & Communication Technology, Consultants']
['7 Feb 2020', 'Melbourne, CBD & Inner Suburbs', '$400 - $500 p.d.', 'Contract/Temp', 'Mining, Resources & Energy, Power Generation & Distribution']
['24 Jan 2020', 'Sunshine Coast', 'Contract/Temp', 'Information & Communication Technology, Business/Systems Analysts']
['5 Feb 2020', 'Melbourne, CBD & Inner Suburbs', '$55k- $60k+ Super', 'Full Tim

['5 Feb 2020', 'Melbourne, Bayside & South Eastern Suburbs', '$37++  per hr NEG', 'Contract/Temp', 'Administration & Office Support, Client & Sales Administration']
['21 Jan 2020', 'Alice Springs & Central Australia', 'Contract/Temp', 'Administration & Office Support, Administrative Assistants']
['30 Jan 2020', 'Melbourne, CBD & Inner Suburbs', '$55 - $76 p.h. + super ', 'Contract/Temp', 'Government & Defence, Government - State']
['16 Jan 2020', 'Brisbane, CBD & Inner Suburbs', 'Contract/Temp', 'Information & Communication Technology, Architects']
['6 Feb 2020', 'Brisbane, CBD & Inner Suburbs', '$31.98 - $47.69 p.h. + Super', 'Contract/Temp', 'Administration & Office Support, Records Management & Document Control']
['24 Jan 2020', 'Melbourne, CBD & Inner Suburbs', 'Full Time', 'Sales, New Business Development']
['30 Jan 2020', 'Brisbane, Bayside & Eastern Suburbs', 'Casual/Vacation', 'Healthcare & Medical, Medical Administration']
['29 Jan 2020', 'Sydney', '$65k - $75k p.a. + super', 

['5 Feb 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', ' + Super', 'Full Time', 'Trades & Services, Electricians']
['6 Feb 2020', 'Gold Coast', 'Full Time', 'Administration & Office Support, Receptionists']
['16 Jan 2020', 'Melbourne, Western Suburbs', 'Contract/Temp', 'Administration & Office Support, Administrative Assistants']
['26 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', '$80k - $110000.00 p.a.', 'Full Time', 'Information & Communication Technology, Product Management & Development']
['6 Feb 2020', 'Melbourne, Eastern Suburbs', '$70-80ph', 'Contract/Temp', 'Information & Communication Technology, Database Development & Administration']
['3 Feb 2020', 'Melbourne, Bayside & South Eastern Suburbs', 'Full Time', 'Healthcare & Medical, Medical Administration']
['6 Feb 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Attractive Package', 'Full Time', 'Insurance & Superannuation, Risk Consulting']
['29 Jan 2020', 'Sydney, North West & Hills District', '$750-$800 da

['9 Feb 2020', 'Perth, CBD, Inner & Western Suburbs', '$28.38ph + Super', 'Contract/Temp', 'Administration & Office Support, Administrative Assistants']
['5 Feb 2020', 'Sydney, Parramatta & Western Suburbs', 'Full Time', 'Trades & Services, Bakers & Pastry Chefs']
['22 Jan 2020', 'Perth, CBD, Inner & Western Suburbs', 'Contract/Temp', 'Information & Communication Technology, Business/Systems Analysts']
['24 Jan 2020', 'Sydney, South West & M5 Corridor', '$90k - $100k p.a.', 'Contract/Temp', 'Accounting, Financial Accounting & Reporting']
['3 Feb 2020', 'Sydney, North Shore & Northern Beaches', 'Full Time', 'Information & Communication Technology, Developers/Programmers']
['31 Jan 2020', 'Melbourne, CBD & Inner Suburbs', 'Full Time', 'Information & Communication Technology, Database Development & Administration']
['7 Feb 2020', 'Perth, CBD, Inner & Western Suburbs', 'Full Time', 'Accounting, Payroll']
['7 Feb 2020', 'Perth, Eastern Suburbs', 'Casual/Vacation', 'Accounting, Bookkeeping &

...

['23 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Full Time', 'Engineering, Project Engineering']
['30 Dec 2019', 'Sydney, CBD, Inner West & Eastern Suburbs', '$55,000 - $69,999 base plus commission', 'Full Time', 'Sales, Account & Relationship Management']
['12 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', '$70,000 - $84,999', 'Full Time', 'Construction, Project Management']
['13 Jan 2020', 'Brisbane, CBD & Inner Suburbs', 'Full Time', 'Administration & Office Support, Administrative Assistants']
['13 Jan 2020', 'Brisbane, CBD & Inner Suburbs', 'Full Time', 'Administration & Office Support, Administrative Assistants']
['15 Jan 2020', 'Melbourne, Bayside & South Eastern Suburbs', 'Contract/Temp', 'Administration & Office Support, Administrative Assistants']
['21 Jan 2020', 'Port Macquarie & Mid North Coast', 'Full Time', 'Community Services & Development, Management']
['15 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Circa $55k + super', 'Full Time', '

['29 Jan 2020', 'Sydney, North Shore & Northern Beaches', '$70,000 - $84,999', 'Full Time', 'Administration & Office Support, Administrative Assistants']
['13 Jan 2020', 'Sydney', 'Industry leading salary packages', 'Full Time', 'Engineering, Project Engineering']
['14 Jan 2020', 'Sydney, Parramatta & Western Suburbs', 'Contract/Temp', 'Call Centre & Customer Service, Customer Service - Call Centre']
['15 Jan 2020', 'West Gippsland & Latrobe Valley', '$45,000 - $54,999', 'Full Time', 'Sales, Sales Representatives/Consultants']
['22 Jan 2020', 'Melbourne', 'Full Time', 'Banking & Financial Services, Analysis & Reporting']
['13 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Full Time', 'Administration & Office Support, PA, EA & Secretarial']
['15 Jan 2020', 'Melbourne, Northern Suburbs', 'Casual/Vacation', 'Trades & Services, Hair & Beauty Services']
['17 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', '$110-130K including super, based on experience', 'Full Time', 'Huma

['14 Jan 2020', 'Broome & Kimberley', '12mth contract, base salary starting $87,655.07', 'Full Time', 'Government & Defence, Government - State']
['14 Jan 2020', 'Brisbane, CBD & Inner Suburbs', '$85,000 - $99,999 plus super, Negotiable on exp', 'Part Time', 'Information & Communication Technology, Testing & Quality Assurance']
['20 Jan 2020', 'Melbourne, Eastern Suburbs', 'Base + Super + Car Allowance + Uncapped Commission', 'Full Time', 'Sales, Account & Relationship Management']
['16 Jan 2020', 'Yarra Valley & High Country', 'Full Time', 'Engineering, Electrical/Electronic Engineering']
['16 Jan 2020', 'Perth, CBD, Inner & Western Suburbs', '$70,000 - $74,999', 'Full Time', 'Government & Defence, Government - Federal']
['13 Jan 2020', 'Sydney, Parramatta & Western Suburbs', 'Full Time', 'Administration & Office Support, Office Management']
['20 Jan 2020', 'ACT', 'Part Time', 'Healthcare & Medical, Psychology, Counselling & Social Work']
['22 Jan 2020', 'Brisbane, CBD & Inner Suburbs

['23 Jan 2020', 'Newcastle, Maitland & Hunter', 'Full Time', 'Healthcare & Medical, Speech Therapy']
['14 Jan 2020', 'Newcastle, Maitland & Hunter', 'Full Time', 'Administration & Office Support, Administrative Assistants']
['13 Jan 2020', 'Perth, CBD, Inner & Western Suburbs', '$70,000 - $74,999', 'Full Time', 'Consulting & Strategy, Strategy & Planning']
['21 Jan 2020', 'ACT', 'Full Time', 'Sales, Sales Representatives/Consultants']
['12 Jan 2020', 'Sydney, North Shore & Northern Beaches', '$65,000 - $74,999', 'Full Time', 'Administration & Office Support, Administrative Assistants']
['23 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', '$100,000 - $139,999', 'Full Time', 'Information & Communication Technology, Developers/Programmers']
['15 Jan 2020', 'Sydney, Ryde & Macquarie Park', 'Full Time', 'Government & Defence, Government - Federal']
['19 Jan 2020', 'Blue Mountains & Central West', 'Full Time', 'Accounting, Financial Managers & Controllers']
['21 Jan 2020', 'Sydney, C

['15 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Competitive salary with not for profit packaging ', 'Part Time', 'Community Services & Development, Aged & Disability Support']
['15 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Competitive salary with not for profit packaging ', 'Part Time', 'Community Services & Development, Management']
['15 Jan 2020', 'Sydney, CBD, Inner West & Eastern Suburbs', 'Competitive salary with not for profit packaging ', 'Part Time', 'Healthcare & Medical, Management']
['20 Jan 2020', 'Melbourne', 'Full Time', 'Information & Communication Technology, Developers/Programmers']
['23 Jan 2020', 'Melbourne, Eastern Suburbs', 'Full Time', 'Human Resources & Recruitment, Consulting & Generalist HR']
['23 Jan 2020', 'Melbourne, Eastern Suburbs', 'Full Time', 'Human Resources & Recruitment, Consulting & Generalist HR']
['22 Jan 2020', 'Melbourne, Western Suburbs', 'Full Time', 'Information & Communication Technology, Help Desk & IT Support']

In [3]:
ms.to_pickle('./jobs.pkl')


NameError: name 'ms' is not defined

In [4]:
ms = pd.read_pickle('./jobs.pkl')

In [7]:
ms.columns.values

array(['Titles', 'Description', 'Salary', 'Info'], dtype=object)

In [97]:
ms['Salary'].value_counts()

A $200 bonus ON TOP of your earnings*    21
$80,000 - $84,999                        13
$20 - $24.99 per hour                     9
$50,000 - $59,999                         8
$60,000 - $69,999                         7
                                         ..
$120k - $135k + super                     1
$40k - $50k p.a. Package                  1
$90k - $110k p.a. + Super                 1
$40,000 - $65,000 + BONUSES               1
Contract or Perm Available - Top $$$      1
Name: Salary, Length: 619, dtype: int64

In [98]:
ms['SalaryCleaned'] = ms['Salary'].map(salaryConvOld)

In [276]:
ms['SalaryCleaned'] = ms['SalaryCleaned'].str.extract('(\d+)').apply(lambda x:x+'000' if len(x)<4 else x)

In [281]:
ms.Description[1]

"Data is exploding The demand for great data analysts and data scientists has never been higher than it is today.\xa0 There simply aren't enough to fill the positions being advertised. The Data School Australia is creating the next generation of great data analysts and data scientists. If you are\xa0passionate about\xa0data,we have the best way to get you into your\xa0new data career. How does it work?The successful applicants will be\xa0immersed\xa0into\u202f4 months of\u202fintensive\u202fclassroom training based on the market-leading technologies, Tableau and Alteryx.\xa0 In addition, you will be trained in all the soft skills needed to become a consultant\xa0from our\xa0team of experts. The classroom training is\u202ffollowed by\u202ffour 6-month client engagements with Australia's leading organisations to give you real-world experience to kick-start your new career. Do I get paid to learn?Yes.\xa0 The Data School offers you a competitive salary while you\xa0study and gain valuable

In [99]:
mss = ms.dropna(axis=0)

In [100]:
mss.shape

(455, 5)

In [102]:
mss = mss[mss.SalaryCleaned.astype('int') >20000]

In [103]:
mss.Info.head()[1]

['Sydney, CBD, Inner West & Eastern Suburbs',
 'Full Time',
 'Information & Communication Technology, Consultants']

In [104]:
##defining a function to remove salary
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

In [105]:
## add info column to description column
def dropNum(x):
    for i in x:
        if hasNumbers(i):
            x.remove(i)
    return x

In [106]:
mss['InfoC'] = mss.Info.map(dropNum)

In [107]:
mss.InfoC.head()[1]

['Sydney, CBD, Inner West & Eastern Suburbs',
 'Full Time',
 'Information & Communication Technology, Consultants']

In [108]:
mss['InfoC'] = mss['InfoC'].map(lambda x:' '.join(x))

In [109]:
mss['Description'] = (mss.Description+mss.InfoC)

In [110]:
mss.SalaryCleaned.max()

'98396'

In [111]:
##really wierd, having extreme values from value_counts() but not other methods
mss['SalaryCleaned'].value_counts().sort_values()

49000      1
164767     1
82909      1
106047     1
114100     1
          ..
60000     31
70000     32
50000     37
55000     38
80000     46
Name: SalaryCleaned, Length: 62, dtype: int64

In [112]:
mss.sort_values('SalaryCleaned',ascending=False,inplace=True)

In [113]:
mss.SalaryCleaned=mss.SalaryCleaned.astype('int')

In [114]:
mss.SalaryCleaned.head(50)

658     98396
657     98396
660     97152
1415    96332
1416    96332
3137    95000
3173    95000
2326    95000
2327    95000
2052    95000
1157    94767
1156    94767
1373    94000
2506    90000
436     90000
344     90000
3028    90000
309     90000
820     90000
1875    90000
863     90000
260     90000
3154    90000
490     90000
3155    90000
186     90000
2645    90000
2546    90000
94      90000
2898    90000
775     90000
789     90000
413     90000
3064    87655
3063    87655
1740    85488
1594    85000
3065    85000
1710    85000
1960    82984
448     82909
2461    80669
2612    80000
2999    80000
1702    80000
2764    80000
2333    80000
674     80000
2551    80000
1549    80000
Name: SalaryCleaned, dtype: int32

In [115]:
mss.drop(mss[mss['SalaryCleaned'] > 200000].index,inplace=True)

In [116]:
mss.iloc[45][2]

'$80,000 - $84,999'

In [117]:
# define a function that accepts text and returns a list of lemmas
def split_into_lemmas(text):
    text = str(text).lower()
    words = TextBlob(text).words
    return [word.lemmatize() for word in words]

In [195]:
X = mss.Description

y = mss.SalaryCleaned

In [197]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [231]:
cv = CountVectorizer(analyzer=split_into_lemmas,ngram_range=(1,2),min_df = 3,max_df=0.35)

In [232]:
tokenize_test(cv)

Features:  3053
Accuracy:  0.6203270855323171


Weight?,Feature
+79228.984,<BIAS>
+6513.334,el
+5642.803,mining
+4769.507,c
+4616.937,leader
+4503.462,governance
+4458.630,program
+3960.370,saas
+3705.668,commercial
+3634.803,architect


["'apply", "'can", "'d", "'ll", "'m", "'re", "'s", "'ve", '0', '0-k', '1', '2', '3', '5-k', '5-stage', '6', 'a/b', 'abilities.adept', 'able', 'aboriginal', 'above', 'above.please', 'absence', 'academic', 'academy', 'accc', 'accelerate', 'accelerated', 'accept', 'accepted', 'access', 'accessible', 'accessory', 'accommodation', 'accomplishment', 'accordance', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'accreditation', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'achievement', 'achiever/high', 'achieving', 'acoustic', 'acquire', 'acquisition', 'act', 'acted', 'action', 'active', 'actively', 'activity', 'acumen', 'acute', 'ad', 'ad-hoc', 'ad-tech', 'adapt', 'adaptable', 'add', 'addition', 'additional', 'additionally', 'address', 'addressed', 'addressing', 'adelaide', 'adept', 'adequate', 'adhere', 'adhered', 'adherence', 'adhoc', 'adjustment', 'admin', 'administering', 'administration', 'administrative', 'administrat

In [192]:
## define function removes salary and numbers inside description
def removeSal(text):
    text = re.sub(r'$.+\b','',text)
    text = re.sub(r'\d+(?!-)','',text)
    return text

In [193]:
mss.Description = mss.Description.map(removeSal)

In [194]:
mss.Description[1]

"Data is exploding The demand for great data analysts and data scientists has never been higher than it is today.\xa0 There simply aren't enough to fill the positions being advertised. The Data School Australia is creating the next generation of great data analysts and data scientists. If you are\xa0passionate about\xa0data,we have the best way to get you into your\xa0new data career. How does it work?The successful applicants will be\xa0immersed\xa0into\u202f months of\u202fintensive\u202fclassroom training based on the market-leading technologies, Tableau and Alteryx.\xa0 In addition, you will be trained in all the soft skills needed to become a consultant\xa0from our\xa0team of experts. The classroom training is\u202ffollowed by\u202ffour 6-month client engagements with Australia's leading organisations to give you real-world experience to kick-start your new career. Do I get paid to learn?Yes.\xa0 The Data School offers you a competitive salary while you\xa0study and gain valuable 

### Question 2

In [233]:
mss.columns

Index(['Titles', 'Description', 'Salary', 'Info', 'SalaryCleaned', 'InfoC'], dtype='object')

In [241]:
# create another column for determining whether title has the word 'data' or not
mss['TitleSort'] = mss.Titles.map(lambda x:'Data Job' if any(y in x.lower() for y in ['data','analys']) else 'Non Data Job')

In [242]:
# class imbalance, need oversampling
mss.TitleSort.value_counts()

Non Data Job    404
Data Job         44
Name: TitleSort, dtype: int64

In [243]:
mss.Titles.head()

658              Business Analyst
657              Business Analyst
660     Project Officer Grade 7/8
1415            Database Engineer
1416            Database Engineer
Name: Titles, dtype: object

In [285]:
X = mss.Description

y = mss.TitleSort

In [286]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X.values.reshape(-1,1), y)

In [253]:
X_resampled.shape, y_resampled.shape

((808, 1), (808,))

In [254]:
#now the class is balanced
y_resampled.value_counts()

Non Data Job    404
Data Job        404
Name: TitleSort, dtype: int64

In [287]:
X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled)

In [258]:
cv = CountVectorizer(analyzer=split_into_lemmas,ngram_range=(1,2),min_df = 3,max_df=0.35)

In [274]:
#using TruncatedSVD to reduce dimensionality, and predict using DecisionTreeClassifier
def tokenize_test2(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    svd = TruncatedSVD(n_components=3000)
    svd.fit_transform(X_train_dtm);
    svd.transform(X_test_dtm)
    lr = DecisionTreeClassifier()
    lr.fit(X_train_dtm.todense(), y_train) 
    y_pred_class = lr.predict(X_test_dtm.todense())
    print('Accuracy: ', lr.score(X_test_dtm.todense(),y_test))
    display(eli5.show_weights(lr,feature_names=vect.get_feature_names()))
    print(vect.get_feature_names())

In [275]:
# score is good but using 'analyst' and 'data' to predict 'data analyst' doesn't seem to be useful
tokenize_test2(cv)


Features:  3945
Accuracy:  0.9554455445544554


Weight,Feature
0.3704,analyst
0.2063,engineer
0.0924,machine
0.0603,'data
0.0563,cat
0.0362,security
0.0274,can
0.0261,level.providing
0.0254,jamie
0.0215,directly


["'about", "'an", "'apply", "'at", "'can", "'customer", "'d", "'data", "'end", "'hands", "'immediate", "'job\\xa0descriptionas", "'ll", "'m", "'our", "'partnering", "'product", "'project", "'re", "'s", "'senior", "'talenza", "'the", "'this", "'top", "'ve", "'want", "'we", "'your", "'•\\xa0ongoing", '0', '0-k', '1', '2', '3', '4-month', '5-k', '6', '6-month', 'abilities.adept', 'abilitystrong', 'able', 'aboriginal', 'above', 'above.please', 'absence', 'academic', 'academy', 'accc', 'accelerate', 'accept', 'acceptable', 'acceptance', 'accepted', 'access', 'accessible', 'accessory', 'accommodation', 'accomplishment', 'accordance', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'accreditation', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'achievement', 'achieving', 'acoustic', 'acquire', 'acquisition', 'across', 'act', 'acting', 'action', 'active', 'actively', 'activities.ability', 'activitiesforster', 'activity', 'actua

In [ ]:
## define function removes salary and numbers inside description


In [283]:
#clean the description so that the self explanatory words are removed.
def removeWords(text):
    text = re.sub(r'data|analyst','',text.lower())
    return text

mss.Description = mss.Description.map(removeWords)

In [288]:
tokenize_test2(cv)

Features:  3996
Accuracy:  0.9504950495049505


Weight,Feature
0.1847,business/systems
0.1105,tool
0.0887,however
0.0742,marketing
0.0730,financials
0.0557,extract
0.0493,cabler
0.0481,range
0.0461,allowancesfree
0.0407,questioning


["'about", "'an", "'apply", "'are", "'at", "'can", "'customer", "'d", "'digital", "'end", "'go", "'hands", "'immediate", "'job\\xa0descriptionas", "'ll", "'m", "'our", "'partnering", "'product", "'project", "'re", "'s", "'senior", "'talenza", "'the", "'this", "'top", "'ve", "'want", "'we", "'•\\xa0ongoing", '0', '0-k', '1', '2', '3', '4-month', '5-k', '6', '6-month', '8', 'a/b', 'a\\xa0', 'abilities.adept', 'abilitystrong', 'able', 'aboriginal', 'about\\xa0', 'above', 'above.please', 'absence', 'academic', 'academy', 'accc', 'accept', 'acceptable', 'acceptance', 'accepted', 'access', 'accessible', 'accomplishment', 'accordance', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'accreditation', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'achievement', 'achieving', 'acoustic', 'acquire', 'acquisition', 'act', 'acting', 'action', 'actioning', 'active', 'actively', 'activities.ability', 'activitiesforster', 'activity', 'a